<a href="https://colab.research.google.com/github/mariddc/Projeto-RL-Highway-gym/blob/main/new_HIGHWAY_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GYM - STABLE BASELINES
##  🏎 Ambiente Highway 🏎

O primeiro passo é instalar as bibliotecas necessárias para o notebook.

In [ ]:
# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/carlosluis/stable-baselines3@75fd27e019aaf4dad612823b4544620df2c47844

In [ ]:
# Instalação de bibliotecas necessárias para visualizar o ambiente no Colab
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

In [ ]:
!pip install moviepy
!pip install --upgrade imageio-ffmpeg

Agora vamos importar o ambiente do _gym_ e diferentes tipos de algoritmos que queremos treinar da biblioteca _stable baselines_. Além disso, também importamos ferramentas para, mais tarde, avaliarmos e visualizarmos o desempenho do agente.

In [4]:
# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import PPO, DQN

# Evaluation
from stable_baselines3.common.evaluation import evaluate_policy 

# Visualization
from gym.wrappers import RecordVideo

/usr/local/lib/python3.7/dist-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


Configurando o display para permitir a visualização dos resultados.

In [5]:
import pyvirtualdisplay

# Cria um display dentro do Colab
_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [6]:
from highway_env.envs.common.graphics import EnvViewer

class RenderWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def render(self, mode="rgb_array"):
        """
        Render the environment.
        Create a viewer if none exists, and use it to render an image.
        :param mode: the rendering mode
        """
        self.rendering_mode = mode

        if self.viewer is None:
            self.viewer = EnvViewer(self)

        self.enable_auto_render = True

        self.viewer.display()

        if not self.viewer.offscreen:
            self.viewer.handle_events()
        if mode == 'rgb_array':
            image = self.viewer.get_image()
            return image

Criamos o ambiente que queremos consultando as possibilidades da biblioteca [_gymnasium_](https://gymnasium.farama.org/environments/third_party_environments/). Depois, escolhemos o algoritmo do [_stable baselines_](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html) que será testado, nesse caso o [PPO](https://openai.com/blog/openai-baselines-ppo/) (Proximal Policy Optimization) e rodamos um episódio inicial.

In [8]:
env = gym.make("racetrack-v0", render_mode="rgb_array")
env = RenderWrapper(env)
env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)

env.metadata['render_fps'] = 30
env.unwrapped.render_mode = "rgb_array"
env.unwrapped.set_record_video_wrapper(env)

env = env 

obs, done = env.reset(seed=53), False

for i in range(150):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

    #done = terminated or truncated

env.close()

/usr/local/lib/python3.7/dist-packages/highway_env/envs/common/observation.py:270: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  grid_shape = np.asarray(np.floor((self.grid_size[:, 1] - self.grid_size[:, 0]) / self.grid_step), dtype=np.int)
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:21: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  

In [9]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('videos/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [10]:
# Setting model
model = PPO('MlpPolicy', env, seed=88, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.7/dist-packages/highway_env/envs/common/observation.py:270: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  grid_shape = np.asarray(np.floor((self.grid_size[:, 1] - self.grid_size[:, 0]) / self.grid_step), dtype=np.int)


In [15]:
# evaluation
eval_env = gym.make("racetrack-v0")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

ValueError: ignored

In [14]:
model.learn(total_timesteps=1000)

ValueError: ignored